# Combine all the actuals and the forecasted data into a single .csv file

In [ ]:
import pandas as pd
import numpy as np
import os
import re
import plotly
import plotly.graph_objs as go
import holidays
from sklearn.metrics import mean_absolute_error, mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt
import itertools

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd = pd.read_parquet('/content/drive/MyDrive/MGT6748_Steelcase/final_clean_data_0317.parquet')

In [ ]:
cd1 = pd.read_csv('/content/drive/MyDrive/MGT6748_Steelcase/frsct_w_tuning.csv', header=0)

In [ ]:
cd2 = pd.read_csv('/content/drive/MyDrive/MGT6748_Steelcase/frsct_w_tuning_inv_sqft.csv', header=0)

In [ ]:
# cd Clean up
cd_cln1 = cd.loc[(cd['Location'] == "In Building") & (cd['report_date'] >= '2022-09-15') & (cd['report_date'] <= '2022-12-31')]

cd_cln2 = cd_cln1.groupby(['report_date','rdc']).agg( inc_inv=('inc', 'sum'),out_inv=('out', 'sum'),total_inv=('total', 'sum'),vol_total=('total_vol', 'sum')).reset_index()

# Unpivot the table to create "metric_value" and "metric_name" columns
melted_df = cd_cln2.melt(id_vars=['report_date', 'rdc'], var_name='metric_name', value_name='metric_value')

cd = melted_df

cd['flag'] = 'Actuals'

In [ ]:
# cd1 Clean up
cd1 = cd1.loc[cd1['predicted_name'].isin(['inc_inv', 'out_inv'])]
cd1['flag'] = 'Predicted'
# Load the data - date and y variable
cd1=cd1[["date",'building', 'predicted_name' ,'predicted_value','flag','rmse']]

# Rename the columns
cd1.columns = ['report_date','rdc','metric_name','metric_value','flag','rmse']

cd1 = cd1.astype(cd.dtypes.to_dict())

<ipython-input-11-96a57f84831b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cd1['flag'] = 'Predicted'


In [ ]:
#cd2 claen up
cd2['predicted_name'] = cd2['predicted_name'].apply(lambda x: 'total_inv' if x.endswith('_RDC_Building') else x)

cd2['predicted_name'] = cd2['predicted_name'].apply(lambda x: 'vol_total' if x.endswith('_Sq_Ft_Used') else x)

cd2['Flag'] = 'Predicted'

# Load the data - date and y variable
cd2=cd2[["date",'building', 'predicted_name' ,'predicted_value','Flag','rmse']]

# Rename the columns
cd2.columns = ['report_date','rdc','metric_name','metric_value','flag','rmse']

cd2.loc[cd2['metric_name'] == 'vol_total', 'metric_value'] = cd2.loc[cd2['metric_name'] == 'vol_total', 'metric_value'] / 0.464

cd2 = cd2.astype(cd.dtypes.to_dict())

In [ ]:
combined_df = pd.concat([cd, cd1, cd2], ignore_index=True)

In [ ]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3730 entries, 0 to 3735
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   report_date   3730 non-null   datetime64[ns]
 1   rdc           3730 non-null   object        
 2   metric_name   3730 non-null   object        
 3   metric_value  3730 non-null   float64       
 4   flag          3730 non-null   object        
 5   rmse          1848 non-null   float64       
 6   sqft_avail    3730 non-null   int64         
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 233.1+ KB


In [ ]:
combined_df = combined_df.loc[~((combined_df['report_date'] == '2022-12-30') & (combined_df['metric_name'] == 'out_inv'))]

In [ ]:
combined_df['metric_value'][combined_df['metric_value'] < 0] = 0

In [ ]:
combined_df.loc[combined_df['metric_name'] == 'vol_total', 'metric_value'] *= 0.464

In [ ]:
combined_df.loc[combined_df['metric_name'] == 'vol_total', 'metric_name'] = 'total_sqft'

In [ ]:
rdc_dict = {'CC': 130000, 'EP': 165000, 'GM': 218274, 'NC': 50000, 'NG': 140000, 'NT': 75000, 'WW': 34789}
combined_df['sqft_avail'] = combined_df['rdc'].map(rdc_dict)

In [ ]:
combined_df.to_csv('/content/drive/MyDrive/MGT6748_Steelcase/final_clean_forecast_data.csv', index=False) 